In [1]:
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QHBoxLayout, QPushButton, QSlider, QLabel, QColorDialog, QSpinBox, QTableWidget, QTableWidgetItem
from PyQt5.QtCore import QTimer, Qt, QRect
from PyQt5.QtGui import QPainter, QColor, QPen
import random
import sys
import json

class Raindrop:
    def __init__(self, x, y, speed_range, density, angle, cloud_id):
        self.x = x + random.randint(-density // 2, density // 2)
        self.y = y
        self.speed = random.uniform(*speed_range)
        self.angle = angle
        self.radius = random.randint(2, 4)
        self.cloud_id = cloud_id

    def fall(self, height):
        self.x += self.angle * self.speed
        self.y += self.speed
        return 0 <= self.x <= 800 and self.y <= height


class Cloud:
    def __init__(self, x, y, width, height, color, speed_range, density, shape, cloud_id):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.color = color
        self.speed_range = speed_range
        self.density = min(density, width)
        self.shape = shape
        self.cloud_id = cloud_id
        self.raindrops = []

    def generate_raindrop(self):
        if random.random() < self.density / 100:
            drop_x = random.randint(self.x, self.x + self.width)
            drop_y = self.y + self.height
            angle = random.uniform(-0.3, 0.3)
            self.raindrops.append(Raindrop(drop_x, drop_y, self.speed_range, self.density, angle, self.cloud_id))

    def update_raindrops(self, height):
        self.raindrops = [drop for drop in self.raindrops if drop.fall(height)]

    def contains_point(self, point):
        return (self.x <= point.x() <= self.x + self.width) and (self.y <= point.y() <= self.y + self.height)


class RainWindow(QWidget):
    def __init__(self, config):
        super().__init__()
        self.setGeometry(100, 100, 800, 600)
        self.clouds = []
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_scene)
        self.timer.start(16)
        self.selected_cloud = None
        self.drag_start_position = None
        self.num_buckets = config['num_buckets']
        self.bucket_width = 800 // self.num_buckets
        self.bucket_height = 150
        self.buckets = [{} for _ in range(self.num_buckets)]

    def add_cloud(self, x, y, width, height, color, speed_range, density):
        cloud_id = len(self.clouds)
        shape = random.choice(["ellipse", "rectangle"])
        self.clouds.append(Cloud(x, y, width, height, color, speed_range, density, shape, cloud_id))

    def mousePressEvent(self, event):
        for cloud in self.clouds:
            if cloud.contains_point(event.pos()):
                self.selected_cloud = cloud
                self.drag_start_position = event.pos()
                break

    def mouseReleaseEvent(self, event):
        if self.selected_cloud and self.drag_start_position:
            delta = event.pos() - self.drag_start_position
            self.selected_cloud.x += delta.x()
            self.selected_cloud.y += delta.y()
        self.selected_cloud = None
        self.drag_start_position = None

    def paintEvent(self, event):
        painter = QPainter(self)

        for cloud in self.clouds:
            painter.setBrush(cloud.color if cloud != self.selected_cloud else QColor(255, 0, 0, 128))
            painter.setPen(Qt.NoPen)
            if cloud.shape == "ellipse":
                painter.drawEllipse(
                    int(cloud.x),
                    int(cloud.y),
                    int(cloud.width),
                    int(cloud.height)
                )
            elif cloud.shape == "rectangle":
                painter.drawRect(
                    int(cloud.x),
                    int(cloud.y),
                    int(cloud.width),
                    int(cloud.height)
                )

        for i in range(self.num_buckets):
            bucket_top = 450
            painter.setBrush(QColor(200, 200, 200))
            painter.setPen(QPen(Qt.black, 3))
            bucket_rect = QRect(i * self.bucket_width, bucket_top, self.bucket_width, self.bucket_height)
            painter.drawRect(bucket_rect)

            painter.setPen(Qt.black)
            painter.drawText(
                bucket_rect.center().x() - 20, bucket_rect.center().y(),
                f"Bucket {i + 1}"
            )

        painter.setBrush(QColor(0, 0, 255))
        for cloud in self.clouds:
            for raindrop in cloud.raindrops:
                painter.drawEllipse(
                    int(raindrop.x - raindrop.radius),
                    int(raindrop.y - raindrop.radius),
                    int(2 * raindrop.radius),
                    int(2 * raindrop.radius)
                )

    def update_scene(self):
        self.buckets = [{} for _ in range(self.num_buckets)]
        for cloud in self.clouds:
            cloud.generate_raindrop()
            for drop in cloud.raindrops:
                if drop.y >= 450:
                    bucket_index = int(drop.x // self.bucket_width)
                    if "count" not in self.buckets[bucket_index]:
                        self.buckets[bucket_index] = {
                            "count": 0,
                            "clouds": set(),
                            "speed_sum": 0,
                            "angle_sum": 0
                        }
                    self.buckets[bucket_index]["count"] += 1
                    self.buckets[bucket_index]["clouds"].add(drop.cloud_id)
                    self.buckets[bucket_index]["speed_sum"] += drop.speed
                    self.buckets[bucket_index]["angle_sum"] += drop.angle
            cloud.update_raindrops(self.height())
        self.repaint()

    def get_bucket_data(self):
        data = []
        for i, bucket in enumerate(self.buckets):
            if "count" in bucket:
                count = bucket["count"]
                clouds = ", ".join(map(str, bucket["clouds"]))
                avg_speed = bucket["speed_sum"] / count if count > 0 else 0
                avg_angle = bucket["angle_sum"] / count if count > 0 else 0
                data.append((i + 1, count, clouds, avg_speed, avg_angle))
        return data


class ControlPanel(QWidget):
    def __init__(self, rain_window):
        super().__init__()
        self.rain_window = rain_window
        self.layout = QVBoxLayout()
        self.setLayout(self.layout)

        self.add_cloud_button = QPushButton("Add Cloud")
        self.add_cloud_button.clicked.connect(self.add_cloud)
        self.layout.addWidget(self.add_cloud_button)

        self.color = QColor(200, 200, 200)

        self.width_label = QLabel("Cloud Width")
        self.layout.addWidget(self.width_label)
        self.width_spinbox = QSpinBox()
        self.width_spinbox.setMinimum(50)
        self.width_spinbox.setMaximum(300)
        self.layout.addWidget(self.width_spinbox)

        self.height_label = QLabel("Cloud Height")
        self.layout.addWidget(self.height_label)
        self.height_spinbox = QSpinBox()
        self.height_spinbox.setMinimum(50)
        self.height_spinbox.setMaximum(200)
        self.layout.addWidget(self.height_spinbox)

        self.speed_label = QLabel("Raindrop Speed Range")
        self.layout.addWidget(self.speed_label)
        self.speed_slider = QSlider(Qt.Horizontal)
        self.speed_slider.setMinimum(1)
        self.speed_slider.setMaximum(10)
        self.speed_slider.setValue(5)
        self.layout.addWidget(self.speed_slider)

        self.density_label = QLabel("Cloud Density")
        self.layout.addWidget(self.density_label)
        self.density_slider = QSlider(Qt.Horizontal)
        self.density_slider.setMinimum(1)
        self.density_slider.setMaximum(100)
        self.density_slider.setValue(50)
        self.layout.addWidget(self.density_slider)

        self.table = QTableWidget()
        self.table.setColumnCount(5)
        self.table.setHorizontalHeaderLabels(["Bucket", "Count", "Cloud IDs", "Avg Speed", "Avg Angle"])
        self.layout.addWidget(self.table)

        self.update_timer = QTimer(self)
        self.update_timer.timeout.connect(self.update_table)
        self.update_timer.start(500)

    def select_color(self):
        color = QColorDialog.getColor()
        if color.isValid():
            self.color = color

    def add_cloud(self):
        x = random.randint(50, 700)
        y = random.randint(50, 200)
        width = self.width_spinbox.value()
        height = self.height_spinbox.value()
        speed_range = (1, self.speed_slider.value())
        density = self.density_slider.value()
        self.rain_window.add_cloud(x, y, width, height, self.color, speed_range, density)

    def update_table(self):
        data = self.rain_window.get_bucket_data()
        self.table.setRowCount(len(data))
        for row, (bucket, count, clouds, avg_speed, avg_angle) in enumerate(data):
            self.table.setItem(row, 0, QTableWidgetItem(str(bucket)))
            self.table.setItem(row, 1, QTableWidgetItem(str(count)))
            self.table.setItem(row, 2, QTableWidgetItem(clouds))
            self.table.setItem(row, 3, QTableWidgetItem(f"{avg_speed:.2f}"))
            self.table.setItem(row, 4, QTableWidgetItem(f"{avg_angle:.2f}"))


if __name__ == '__main__':
    with open("config.json", "r") as config_file:
        config = json.load(config_file)

    app = QApplication(sys.argv)

    main_window = QWidget()
    main_layout = QVBoxLayout()
    main_window.setLayout(main_layout)

    rain_window = RainWindow(config)
    control_panel = ControlPanel(rain_window)

    rain_window.setMinimumHeight(500)
    control_panel.setMinimumHeight(100)

    top_layout = QVBoxLayout()
    top_layout.addWidget(rain_window)

    middle_layout = QVBoxLayout()
    middle_layout.addWidget(control_panel)

    bottom_layout = QVBoxLayout()
    bottom_layout.addWidget(control_panel.table)

    main_layout.addLayout(top_layout)
    main_layout.addLayout(middle_layout)
    main_layout.addLayout(bottom_layout)

    main_window.show()
    sys.exit(app.exec_())


SystemExit: 0

C:\Users\Bodry\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
